Ce notebook vsie à effectuer les prétraitements nécessaires et qui permettent d'aboutir aux datasets qui seront utilisés dans les deux notebooks suivants sur l'exploration de modèle et l'optimisation du meilleur modèle apparent. Pour ce jeu de données, on s'intéressera à un modèle de régression pour prédire l'imdb_score d'un film ou d'une série. 

Les prétraitements touchent principalement à la transformation des variables catégorielles : remapping et one-hot encoding.

A la fin de ce notebook, les datasets prétraités sont sauvegardés.

# Prétraitements

## I. Chargement des données

In [1]:
#Importation des librairies

import pandas as pd

In [2]:
#Chargement des données 
original_df = pd.read_csv('titles.csv')

# Affichage des 10 premières lignes du jeu de données
original_df.head(10)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.601,NaN
1,tm82169,Rocky,MOVIE,"When world heavyweight boxing champion, Apollo...",1976,PG,119,"['drama', 'sport']",['US'],NaN,tt0075148,8.1,588100.0,106.361,7.782
2,tm17823,Grease,MOVIE,Australian good girl Sandy and greaser Danny f...,1978,PG,110,"['romance', 'comedy']",['US'],NaN,tt0077631,7.2,283316.0,33.160,7.406
3,tm191099,The Sting,MOVIE,A novice con man teams up with an acknowledged...,1973,PG,129,"['crime', 'drama', 'comedy', 'music']",['US'],NaN,tt0070735,8.3,266738.0,24.616,8.020
4,tm69975,Rocky II,MOVIE,After Rocky goes the distance with champ Apoll...,1979,PG,119,"['drama', 'sport']",['US'],NaN,tt0079817,7.3,216307.0,75.699,7.246
5,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'comedy']",['GB'],NaN,tt0071853,8.2,547292.0,20.964,7.804
6,tm17249,Animal House,MOVIE,"At a 1962 College, Dean Vernon Wormer is deter...",1978,R,109,['comedy'],['US'],NaN,tt0077975,7.4,123611.0,17.372,7.020
7,ts22164,Monty Python's Flying Circus,SHOW,A British sketch comedy series with the shows ...,1969,TV-14,30,"['comedy', 'european']",['GB'],4.0,tt0063929,8.8,75654.0,24.773,8.258
8,tm70993,Life of Brian,MOVIE,"Brian Cohen is an average young Jewish man, bu...",1979,R,94,['comedy'],['GB'],NaN,tt0079470,8.0,405022.0,19.787,7.762
9,tm16479,White Christmas,MOVIE,Two talented song-and-dance men team up after ...,1954,NaN,120,"['comedy', 'music', 'romance']",['US'],NaN,tt0047673,7.5,46586.0,11.598,7.200


In [3]:
print("Le jeu de données contient :",original_df.shape[0], " enregistrements et ", original_df.shape[1], " colonnes.")
print("Les colonnes sont : ", original_df.columns)

Le jeu de données contient : 6137  enregistrements et  15  colonnes.
Les colonnes sont :  Index(['id', 'title', 'type', 'description', 'release_year',
       'age_certification', 'runtime', 'genres', 'production_countries',
       'seasons', 'imdb_id', 'imdb_score', 'imdb_votes', 'tmdb_popularity',
       'tmdb_score'],
      dtype='object')


Description des colonnes : 

id: The title ID on JustWatch.

title: The name of the title.

show_type: TV show or movie.

description: A brief description.

release_year: The release year.

age_certification: The age certification.

runtime: The length of the episode (SHOW) or movie.

genres: A list of genres.

production_countries: A list of countries that - produced the title.

seasons: Number of seasons if it's a SHOW.

imdb_id: The title ID on IMDB.

imdb_score: Score on IMDB.

imdb_votes: Votes on IMDB.

tmdb_popularity: Popularity on TMDB.

tmdb_score: Score on TMDB.

In [4]:
#Affichage de statistiques descriptives pour les descripteurs numériques 
original_df.describe()

,release_year,runtime,seasons,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
count,6137.000000,6137.000000,2306.000000,5669.000000,5.653000e+03,6061.000000,5885.000000
mean,2017.371843,76.381946,2.106678,6.540942,2.115029e+04,19.267196,6.633194
std,6.603620,39.086828,2.716844,1.135944,9.254225e+04,51.291407,1.251610
min,1945.000000,0.000000,1.000000,1.500000,5.000000e+00,0.009442,0.500000
25%,2017.000000,44.000000,1.000000,5.800000,5.170000e+02,3.381000,6.000000
50%,2019.000000,80.000000,1.000000,6.600000,2.095000e+03,7.580000,6.791000
75%,2021.000000,105.000000,2.000000,7.300000,8.884000e+03,16.523000,7.400000
max,2023.000000,225.000000,44.000000,9.600000,2.684317e+06,1078.637000,10.000000


## II. Nettoyage des données

In [5]:
# Suppression des colonnes inutiles pour une première exploration 
colonnes_a_supprimer = ['id', 'description', 'imdb_id', 'imdb_votes', 'tmdb_popularity', 'tmdb_score']
df = original_df.drop(columns=colonnes_a_supprimer)

In [6]:
# Suppression des données manquantes pour le descripteur imdb_score
df = df.dropna(subset=['imdb_score'])
print('Il reste ', df.shape[0], " enregistrements.")
df.describe()

Il reste  5669  enregistrements.


,release_year,runtime,seasons,imdb_score
count,5669.000000,5669.000000,2143.000000,5669.000000
mean,2017.267772,77.945317,2.151657,6.540942
std,6.689366,38.890644,2.748713,1.135944
min,1954.000000,0.000000,1.000000,1.500000
25%,2017.000000,45.000000,1.000000,5.800000
50%,2019.000000,83.000000,1.000000,6.600000
75%,2021.000000,105.000000,2.000000,7.300000
max,2023.000000,225.000000,44.000000,9.600000


## III. Transformations des variables catégorielles

In [7]:
# Affichage du type de chaque colonne
types_de_colonnes = df.dtypes
print(types_de_colonnes)

title                    object
type                     object
release_year              int64
age_certification        object
runtime                   int64
genres                   object
production_countries     object
seasons                 float64
imdb_score              float64
dtype: object


Les descripteurs type, release_year, age_certification, genres et production_countries ne sont pas des variables numériques.

### III.1. Transformation de 'age_certification' 

In [8]:
print("Nombre de valeurs manquantes pour la variable : ", df['age_certification'].isnull().sum())

Nombre de valeurs manquantes pour la variable :  2460


In [9]:
# Affichage des valeurs uniques de la colonne
print("Valeurs uniques de la colonne 'age_certification' : ", df['age_certification'].unique())

Valeurs uniques de la colonne 'age_certification' :  ['PG' 'R' 'TV-14' nan 'TV-MA' 'TV-PG' 'PG-13' 'TV-Y' 'TV-Y7' 'TV-G' 'G'
 'NC-17' 'TV-Y7-FV']


Pour mieux comprendre ce dont il s'agit dans cette colonne, on décide de remapper les valeurs de la certification d'âge. On recherche sur internet à quelle catégorie d'âge, plus explicite, ces codes correspondent.
Remarque : les classes d'âge varient parfois en fonction des pays où le film est diffusé. Cependant, on s'appuie sur le référentiel américain et notamment tvguidelines.org. Le but du remapping étant la simplification, certaines nuances ont été également omises dans les nouvelles valeurs du remapping. Par exemple, 'TV-Y7' et'TV-Y7-FV'ont été toutes deux remappées en '7+' bien que la seconde mention indique la présence de "fantasy violence". 

In [10]:
# Création d'un dictionnaire de mapping des valeurs
mapping_valeurs = {'TV-MA': '17+', 'PG': '12+', 'R':'17+', 'TV-14':'13+',  'TV-PG':'7+', 'PG-13':'13+', 'TV-Y':'0+', 'TV-Y7':'7+', 'TV-G':'0+', 'G':'0+',
 'NC-17':'17+', 'TV-Y7-FV':'7+'}

# Application du remapping à la colonne age_certification
df['age_certification'] = df['age_certification'].map(mapping_valeurs)

In [11]:
df['age_certification'].unique()

array(['12+', '17+', '13+', nan, '7+', '0+'], dtype=object)

### III.2. Transformation de 'genres'

Pour simplifier l'entraînement du modèle, on simplifie la colonne 'genres' en ne conservant qu'un seul genre, le 1er de la liste, pour chacun des enregistrements.

In [12]:
#On vérifie d'baord que la variable 'genres' n'a pas de valeurs manquantes et que tous les enregistrements possèdent bien au moins un genre.
print("Nombre de valeurs manquantes : ", df['genres'].isnull().sum())

Nombre de valeurs manquantes :  0


In [13]:
# On crée une nouvelle colonne 'genre' qui ne retient que le 1er genre de la colonne 'genres' pour chacune des lignes 
df['genre'] = df['genres'].apply(lambda x: x.replace("[","").replace("]","").replace("'",'').split(",")[0])

In [14]:
#Suppression de la colonne 'genres'. On ne garde que la nouvelle colonne 'genre' créée
df.drop(columns='genres', inplace=True)

In [15]:
df['genre'].unique()

array(['drama', 'romance', 'crime', 'fantasy', 'comedy', 'documentation',
       'thriller', 'action', 'animation', 'family', 'reality', 'scifi',
       'western', 'horror', 'war', 'music', 'history', '', 'sport'],
      dtype=object)

On a bien des catégories de genres de film. On remarque que l'on a un élément "vide" (''). On va supprimer les films n'ayant pas de genre spécifié. 

In [16]:
#On veut supprimer les lignes qui n'ont pas de genre spécifié
print("Taille du df avant suppression : ",df.shape)

valeur = ''

# On compte le nombre de lignes qui n'ont pas de genre spécifié
nombre_de_lignes = (df['genre'] == valeur).value_counts().get(True, 0)
print(f"Nombre de lignes qui n'ont pas de genre spécifié dans la colonne 'genre' : {nombre_de_lignes}")

# Création d'un masque booléen pour identifier les lignes à supprimer
masque = df['genre'] == valeur

# Suppression des lignes correspondantes en utilisant le masque créé
df = df[~masque]

# réindexation du jeu de données si nécessaire
df.reset_index(drop=True, inplace=True)

print("Taille du df après suppression : ",df.shape)

Taille du df avant suppression :  (5669, 9)
Nombre de lignes qui n'ont pas de genre spécifié dans la colonne 'genre' : 5
Taille du df après suppression :  (5664, 9)


In [17]:
#On vérifie les catégories de genre
df['genre'].unique()

array(['drama', 'romance', 'crime', 'fantasy', 'comedy', 'documentation',
       'thriller', 'action', 'animation', 'family', 'reality', 'scifi',
       'western', 'horror', 'war', 'music', 'history', 'sport'],
      dtype=object)

### III.3. Transformation de 'production_countries' 

In [18]:
df['production_countries'].unique()

array(["['US']", "['GB']", "['EG']", "['IN']", "['DE']", "['IN', 'SU']",
       "['EG', 'DZ']", "['CA', 'FR', 'LB']", "['US', 'GB']",
       "['LB', 'FR']", "['JP']", "['AR']", "['EG', 'FR']", "['JP', 'US']",
       "['US', 'CA']", "['GB', 'US']", "['FR']",
       "['IE', 'GB', 'DE', 'FR']", "['AU']",
       "['ET', 'DE', 'GH', 'GB', 'US', 'BF']", "['HK']", "['MX']",
       "['CN', 'HK']", "['DE', 'US']", "['ES', 'AR']",
       "['DE', 'PS', 'US', 'FR']", "['LB', 'BE', 'FR', 'NO']",
       "['SU', 'IN']", "['IT', 'LB', 'BE', 'FR']", "['LB']",
       "['NZ', 'US']", "['GB', 'CA', 'US', 'JP']", "['CA']",
       "['US', 'DE']", "['CA', 'US']", "['US', 'CA', 'DE']", "['DK']",
       "['DE', 'HU', 'GB', 'US']", "['FR', 'DE', 'GB', 'CA']", "['CO']",
       "['US', 'CZ']", "['DE', 'NZ', 'US']", "['IT']", "['IE', 'US']",
       "['GB', 'US', 'CA']", "['MX', 'US']", "['TW']", "['KR']",
       "['IN', 'US']", "['ES', 'GB', 'US']", "['US', 'FR', 'CA']",
       "['CN']", "['JP', 'HK']", "['ES', 'G

De la même manière que pour la variable genres, on ne va conserver que le 1er pays indiqué dans la liste des pays producteurs du film ou de la série. 

In [19]:
# On crée une nouvelle colonne 'production_country' qui ne retient que le 1er pays mentionné de la colonne 'production_countries' pour chacune des lignes 
df['production_country'] = df['production_countries'].apply(lambda x: x.replace("[","").replace("]","").replace("'",'').split(",")[0])

In [20]:
df.drop(columns='production_countries', inplace=True)

In [21]:
df['production_country'].unique()

array(['US', 'GB', 'EG', 'IN', 'DE', 'CA', 'LB', 'JP', 'AR', 'FR', 'IE',
       'AU', 'ET', 'HK', 'MX', 'CN', 'ES', 'SU', 'IT', 'NZ', 'DK', 'CO',
       'TW', 'KR', 'RU', 'NG', '', 'PS', 'TR', 'MY', 'PH', 'ZA', 'MA',
       'SE', 'SG', 'KE', 'NO', 'CL', 'SA', 'BR', 'ID', 'IS', 'IL', 'PL',
       'FI', 'CD', 'RO', 'AE', 'BE', 'NL', 'UA', 'QA', 'GL', 'AT', 'BY',
       'JO', 'VN', 'TN', 'TH', 'KH', 'CH', 'CU', 'UY', 'CZ', 'PE', 'PR',
       'KW', 'IR', 'PY', 'PK', 'HU', 'IQ', 'BD', 'TZ', 'CM', 'KG', 'LU',
       'SN', 'BT', 'PT', 'AO', 'GH', 'ZW', 'MW', 'GT', 'MU', 'BG', 'AF',
       'DO', 'PA', 'IO', 'FO'], dtype=object)

In [22]:
#On veut supprimer les lignes qui n'ont pas de production_country spécifié
print("Taille du df avant suppression : ",df.shape)

valeur = ''

# On compte le nombre de lignes qui n'ont pas de production_country spécifié
nombre_de_lignes = (df['production_country'] == valeur).value_counts().get(True, 0)
print(f"Nombre de lignes qui n'ont pas de genre spécifié dans la colonne 'production_country' : {nombre_de_lignes}")

# Création d'un masque booléen pour identifier les lignes à supprimer
masque = df['production_country'] == valeur

# Suppression des lignes correspondantes en utilisant le masque créé
df = df[~masque]

# Réindexation du jeu de données si nécessaire
df.reset_index(drop=True, inplace=True)

print("Taille du df après suppression : ",df.shape)

Taille du df avant suppression :  (5664, 9)
Nombre de lignes qui n'ont pas de genre spécifié dans la colonne 'production_country' : 128
Taille du df après suppression :  (5536, 9)


### III.4. Transformation de 'type'

In [23]:
print(df['type'].unique())
print("Nombre de valeurs nulles pour la variable 'type' : ", df['type'].isnull().sum())

['MOVIE' 'SHOW']
Nombre de valeurs nulles pour la variable 'type' :  0


Il n'y a que deux valeurs possibles pour la variable type. On n'opère pas plus de transformation pour ce descripteur.

### III.5. Remapping de release_year et runtime

Pour faciliter le one hot encoding sur ces deux variables qui prennent un grand nombre de valeurs, on remappe ces dernières en décennies et tranches de 10 minutes.

In [24]:
df['release_year'].describe()

count    5536.000000
mean     2017.197616
std         6.741180
min      1954.000000
25%      2017.000000
50%      2019.000000
75%      2021.000000
max      2023.000000
Name: release_year, dtype: float64

In [25]:
#Remapping de release_year

# Création d'une nouvelle colonne decade qui prend la résultat de la division entière de release_year et la multiplie par 10
df['decade'] = (df['release_year'] // 10) * 10

#Vérification des valeurs de la colonne decade
df['decade'].unique()
df.drop(columns='release_year', inplace=True)

In [26]:
df['runtime'].unique()

array([119, 110, 129,  91, 109,  30,  94, 120, 112, 102,  77,  78, 141,
       152,  97,  43, 150, 138, 162, 163, 131, 186, 103, 100, 124, 133,
       142,  93, 153,  24, 114, 101,  95,  98, 104, 106, 161,  10, 116,
        84, 107,  72, 108, 144,  14, 135, 174,  87, 134, 160, 172, 159,
       170, 148, 105, 151,  48,  23,  22,  99,  46,  25,  32,  26, 139,
       125, 136,  21,  75,  45,  20,  40,  88,  19,  55, 126, 123, 158,
       206, 177, 147,  29, 165, 146,  59, 175,  58, 149, 145, 156,  50,
        90,  56,  28, 178,  44, 179, 201, 117,  47,  96, 132, 121,  27,
        49, 118,  85, 187, 130,  82,  92,  64, 157, 115, 189,  60, 168,
        18,  42,   9,  89,  37, 155, 137,   6, 166, 113, 122, 182,  80,
        86, 210, 167,  15,  71, 224, 127, 193,  11,  61, 213, 183,  67,
         7,   0, 188,  66, 169,  69,  57,  79, 225, 173,  73, 143,  65,
        54,  53,  17, 154,  83,  81,  16,  68, 128,  63,   3, 140, 111,
         2,  76,  52,  74,  12,  70, 180,  41,   4,  51,  62,  3

Pour le moment, on considère la variable runtime comme numérique, bien que savoir s'il faut la considérer comme catégorielle ou numérique soit ambigue. Par la suite, on pourra toujours remapper les valeurs de runtime si les considérer comme numériques ne s'avère pas très pertinent ou moins pertinent que de les considérer comme catégorielles. 

## IV. Séparation du dataset en deux : movie/série 

Afin d'exploiter la variable seasons pour laquelle nous avons toujours une valeur manquante lorsqu'il s'agit de film, on divise le jeu de données pour en créer deux : l'un comportant les films et donc sans le descripteur 'seasons' et l'autre contenant les séries et donc avec le descripteur 'seasons'. 

In [27]:
# Création de deux nouveaux jeu de données en fonction de la valeur de 'type'
df_movies = df.loc[df['type'] == 'MOVIE']
df_series = df.loc[df['type'] == 'SHOW']
df_movies.drop(columns=['seasons', 'type'], inplace=True)
df_series.drop(columns='type', inplace=True)

C:\Users\guilts0300\AppData\Local\Temp\ipykernel_23808\2842985520.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies.drop(columns=['seasons', 'type'], inplace=True)
C:\Users\guilts0300\AppData\Local\Temp\ipykernel_23808\2842985520.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_series.drop(columns='type', inplace=True)


In [37]:
print("Dimensions de df entier : ",df.shape)
print("Dimensions de df_movies : ", df_movies.shape)
print("Dimensions de df_series : ", df_series.shape)

Dimensions de df entier :  (5536, 9)
Dimensions de df_movies :  (3423, 7)
Dimensions de df_series :  (2113, 8)


In [31]:
print("Nombre de valeurs manquantes de la variable 'seasons' dans df_series : ", df_series['seasons'].isnull().sum())

Nombre de valeurs manquantes de la variable 'seasons' dans df_series :  0


In [33]:
#On vérifie que df_series ne contient pas de valeurs manquantes 
df_series.isnull().sum()

title                   0
age_certification     204
runtime                 0
seasons                 0
imdb_score              0
genre                   0
production_country      0
decade                  0
dtype: int64

df_series contient des valeurs manquantes pour la variable age_certification. On va donc créer 2 sous jeux de données : l'un ne contenant pas l'attribut age_certification, permettant ainsi de garder le plus d'enregistrements possibles et l'autre contenant la variable age_certification pour laquelle il n'y a pas de valeurs manquantes (jeu de données de taille plus petite que le premier).

In [42]:
df_series_wo_age=df_series.drop(columns='age_certification') #sous jeu de données sans (without=wo) le descripteur age_certification
df_series_w_age=df_series.dropna(subset=['age_certification']) #sous jeu de données avec la variable age_certification mais sans les valeurs manquantes

In [40]:
df_series_w_age.isnull().sum()

title                 0
age_certification     0
runtime               0
seasons               0
imdb_score            0
genre                 0
production_country    0
decade                0
dtype: int64

In [43]:
#On effectue la même chose pour df_movies 
df_movies.isnull().sum()

df_movies_wo_age=df_movies.drop(columns='age_certification') #sous dataset sans le descripteur age_certification
df_movies_w_age=df_movies.dropna(subset=['age_certification']) #sous dataset avec la variable age_certification sans NA

Les 4 datasets n'ont désormais plus de valeurs manquantes.

## V. One-hot encodage 

In [49]:
# One-hot encoding sur les variables catégorielles
df_movies_w_age_encoded = pd.get_dummies(df_movies_w_age, columns=['age_certification', 'genre', 'production_country', 'decade'])
df_series_w_age_encoded = pd.get_dummies(df_series_w_age, columns=['age_certification', 'genre', 'production_country', 'decade'])
df_movies_wo_age_encoded = pd.get_dummies(df_movies_wo_age, columns=['genre', 'production_country', 'decade'])
df_series_wo_age_encoded = pd.get_dummies(df_series_wo_age, columns=['genre', 'production_country', 'decade'])

# Sauvegarde des datasets

In [50]:
df_movies_w_age_encoded.to_csv('df_movies_w_age_encoded.csv', index=False)
df_series_w_age_encoded.to_csv('df_series_w_age_encoded.csv', index=False)
df_movies_wo_age_encoded.to_csv('df_movies_wo_age_encoded.csv', index=False)
df_series_wo_age_encoded.to_csv('df_series_wo_age_encoded.csv', index=False)